# Photometric Filter Profiles

In [5]:
import os
import numpy as np
import scipy as sp
import scipy.io.idl
import astropy
import astropy.table
import astropy.io.fits
import astropy.units

import matplotlib.pyplot as plt

import lezargus

%matplotlib Qt

In [ ]:
lezargus.initialize.initialize_logging_outputs()
# Making sure that the output directory exists.
os.makedirs("./products/photometric_filters/", exist_ok=True)

## Johnson UBV Filters
Johnson UBV filter transmission curves below are validated by [stsynphot](https://stsynphot.readthedocs.io/en/latest/stsynphot/appendixb_nonhst.html#stsynphot-appendixb-nonhst) which in turn gets it from [2006AJ....131.1184M](https://ui.adsabs.harvard.edu/abs/2006AJ....131.1184M/abstract). These filters are in the photon counting form so we need to convert it.

In [3]:
# Reading the file.
johnson_filename = "./base/photometric_filters/filter_johnson.dat"
johnson_column_names = [
    "U_wave",
    "U_trans",
    "B_wave",
    "B_trans",
    "V_wave",
    "V_trans",
]
johnson_data = astropy.table.Table.read(
    "./base/photometric_filters/filter_johnson.dat",
    format="ascii.no_header",
    names=johnson_column_names,
)

# We need to operate on all filters.
johnson_filter_names = ["U", "B", "V"]
for filterdex in johnson_filter_names:
    # Loading the filter data.
    # We only need the data that is relevant to our filter tag. We skip over
    # any "masked" data.
    wavetag = f"{filterdex}_wave"
    transtag = f"{filterdex}_trans"
    masked_wavelength = np.ma.asarray(johnson_data[wavetag])
    masked_transmission = np.ma.asarray(johnson_data[transtag])
    wavelength = np.array(masked_wavelength[~masked_wavelength.mask])
    transmission = np.array(masked_transmission[~masked_transmission.mask])

    # We reduce any extra dimensionality.
    wavelength = np.ravel(wavelength)
    transmission = np.ravel(transmission)

    # The wavelength is in Angstroms, we convert it to meters.
    wavelength_meter = wavelength / 1e10

    # We load the filter, leverging the Lezargus container to do the
    # conversion for us.
    filter_container = lezargus.library.container.PhotometricVegaFilter.from_photon_transmission(
        wavelength=wavelength_meter,
        photon_transmission=transmission,
        wavelength_unit="m",
    )

    # The actual data we need to save.
    wavelength_column = astropy.table.Column(
        filter_container.wavelength, name="wavelength", unit="m", dtype=float
    )
    transmission_column = astropy.table.Column(
        filter_container.transmission_energy,
        name="transmission",
        unit="",
        dtype=float,
    )

    table_columns = [wavelength_column, transmission_column]
    johnson_table = astropy.table.Table(table_columns)
    # Too much precision makes unnessary large file sizes.
    table_filename = (
        f"./products/photometric_filters/filter_Johnson_{filterdex}.dat"
    )
    johnson_table.write(
        table_filename,
        format="ascii.mrt",
        formats={keydex: "%.7e" for keydex in johnson_table.keys()},
        overwrite=True,
    )

FileNotFoundError: [Errno 2] No such file or directory: './products/photometric_filters/filter_Johnson_U.dat'

## TYCHO-2 BT VT Filters
Tycho-2 BT and VT filter transmission curves are validated by the [Filter Profile service](https://svo2.cab.inta-csic.es/theory/fps/) which in turn gets it from [2015PASP..127..102M]. These filters are in the energy counting form so we need to supply it as is.

In [ ]:
# There are two files.
tycho2_bt_filename = "./base/photometric_filters/filter_TYCHO2_BT.dat"
tycho2_vt_filename = "./base/photometric_filters/filter_TYCHO2_VT.dat"
tycho2_filenames = [tycho2_bt_filename, tycho2_vt_filename]

for filedex in tycho2_filenames:
    tycho2_data = astropy.table.Table.read(
        filedex, format="ascii.no_header", names=["wave", "trans"]
    )
    # We only need the data that is relevant to our filter tag. We skip over
    # any "masked" data.
    wavelength = np.array(tycho2_data["wave"])
    transmission = np.array(tycho2_data["trans"])

    # The wavelength provided by the Tycho2 files are in Angstroms, we convert
    # to meters.
    wavelength_meter = lezargus.library.conversion.convert_units(
        value=wavelength, value_unit="AA", result_unit="m"
    )

    # We load the filter, leveraging the Lezargus container.
    filter_container = lezargus.library.container.PhotometricVegaFilter.from_energy_transmission(
        wavelength=wavelength_meter,
        energy_transmission=transmission,
        wavelength_unit="m",
    )

    # The actual data we need to save.
    wavelength_column = astropy.table.Column(
        filter_container.wavelength, name="wavelength", unit="m", dtype=float
    )
    transmission_column = astropy.table.Column(
        filter_container.transmission_energy,
        name="transmission",
        unit="",
        dtype=float,
    )

    table_columns = [wavelength_column, transmission_column]
    tycho2_table = astropy.table.Table(table_columns)
    # Too much precision makes unnessary large file sizes.
    basename = lezargus.library.path.get_filename_without_extension(
        pathname=filedex
    )
    table_filename = f"./products/photometric_filters/{basename}.dat"
    tycho2_table.write(
        table_filename,
        format="ascii.mrt",
        formats={keydex: "%.7e" for keydex in tycho2_table.keys()},
        overwrite=True,
    )

## GAIA BP, G, RP Filters
GAIA BP, G, and RP filter transmission curves are validated by the [Filter Profile service](https://svo2.cab.inta-csic.es/theory/fps/) which in turn gets it from [2021A&A...649A...3R] and [2023A&A...674A...6S]. These filters are in the photon counting form so we need to convert it.

In [ ]:
# There are four files.
gaia_g_filename = "./base/photometric_filters/filter_GAIA_G.dat"
gaia_bp_filename = "./base/photometric_filters/filter_GAIA_BP.dat"
gaia_rp_filename = "./base/photometric_filters/filter_GAIA_RP.dat"
gaia_rvs_filename = "./base/photometric_filters/filter_GAIA_RVS.dat"
gaia_filenames = [
    gaia_g_filename,
    gaia_bp_filename,
    gaia_rp_filename,
    gaia_rvs_filename,
]

for filedex in gaia_filenames:
    gaia_data = astropy.table.Table.read(
        filedex, format="ascii.no_header", names=["wave", "trans"]
    )
    # We only need the data that is relevant to our filter tag. We skip over
    # any "masked" data.
    wavelength = np.array(gaia_data["wave"])
    transmission = np.array(gaia_data["trans"])

    # The wavelength provided by the Tycho2 files are in Angstroms, we convert
    # to meters.
    wavelength_meter = lezargus.library.conversion.convert_units(
        value=wavelength, value_unit="AA", result_unit="m"
    )

    # We load the filter, leveraging the Lezargus container.
    filter_container = lezargus.library.container.PhotometricVegaFilter.from_photon_transmission(
        wavelength=wavelength_meter,
        photon_transmission=transmission,
        wavelength_unit="m",
    )

    # The actual data we need to save.
    wavelength_column = astropy.table.Column(
        filter_container.wavelength, name="wavelength", unit="m", dtype=float
    )
    transmission_column = astropy.table.Column(
        filter_container.transmission_energy,
        name="transmission",
        unit="",
        dtype=float,
    )

    table_columns = [wavelength_column, transmission_column]
    gaia_table = astropy.table.Table(table_columns)
    # Too much precision makes unnessary large file sizes.
    basename = lezargus.library.path.get_filename_without_extension(
        pathname=filedex
    )
    table_filename = f"./products/photometric_filters/{basename}.dat"
    gaia_table.write(
        table_filename,
        format="ascii.mrt",
        formats={keydex: "%.7e" for keydex in gaia_table.keys()},
        overwrite=True,
    )

## 2MASS JHK_s Filters
2MASS JHK_s filter transmission curves below are validated by [stsynphot](https://stsynphot.readthedocs.io/en/latest/stsynphot/appendixb_nonhst.html#stsynphot-appendixb-nonhst) which in turn gets it from the [2MASS webpage](https://irsa.ipac.caltech.edu/data/2MASS/docs/releases/allsky/doc/sec6_4a.html) from the main source [2003AJ....126.1090C](https://ui.adsabs.harvard.edu/abs/2003AJ....126.1090C/abstract). As stated by these authors, these curves *are designed to be integrated directly over stellar spectra in $F_\lambda$ form, in order to calculate synthetic photometric magnitudes. The QE-based component was converted to yield photon-counting RSRs by multiplying by wavelength and renormalized, as described by [Bessel (2000)](https://ui.adsabs.harvard.edu/abs/2000PASP..112..961B/abstract).* As such, these filters are in the photon counting form so we need to convert it.

In [ ]:
# There are three files.
mass2_j_filename = "./base/photometric_filters/filter_2MASS_J.dat"
mass2_h_filename = "./base/photometric_filters/filter_2MASS_H.dat"
mass2_ks_filename = "./base/photometric_filters/filter_2MASS_Ks.dat"
mass2_filenames = [mass2_j_filename, mass2_h_filename, mass2_ks_filename]

for filedex in mass2_filenames:
    mass2_data = astropy.table.Table.read(
        filedex, format="ascii.no_header", names=["wave", "trans"]
    )
    # We only need the data that is relevant to our filter tag. We skip over
    # any "masked" data.
    wavelength = np.array(mass2_data["wave"])
    transmission = np.array(mass2_data["trans"])

    # The wavelength provided by the 2MASS files are in micrometers, we convert
    # to meters.
    wavelength_meter = lezargus.library.conversion.convert_units(
        value=wavelength, value_unit="um", result_unit="m"
    )

    # We load the filter, leveraging the Lezargus container to do the
    # conversion for us.
    filter_container = lezargus.library.container.PhotometricVegaFilter.from_photon_transmission(
        wavelength=wavelength_meter,
        photon_transmission=transmission,
        wavelength_unit="m",
    )

    # The actual data we need to save.
    wavelength_column = astropy.table.Column(
        filter_container.wavelength, name="wavelength", unit="m", dtype=float
    )
    transmission_column = astropy.table.Column(
        filter_container.transmission_energy,
        name="transmission",
        unit="",
        dtype=float,
    )

    table_columns = [wavelength_column, transmission_column]
    mass2_table = astropy.table.Table(table_columns)
    # Too much precision makes unnessary large file sizes.
    basename = lezargus.library.path.get_filename_without_extension(
        pathname=filedex
    )
    table_filename = f"./products/photometric_filters/{basename}.dat"
    mass2_table.write(
        table_filename,
        format="ascii.mrt",
        formats={keydex: "%.7e" for keydex in mass2_table.keys()},
        overwrite=True,
    )

## WISE 1234 Filters
WISE 1, 2, 3, and 4 filter transmission curves below are taken from the [WISE webpage](https://wise2.ipac.caltech.edu/docs/release/allsky/expsup/sec4_4h.html) from the main source [2010AJ....140.1868W](https://ui.adsabs.harvard.edu/abs/2010AJ....140.1868W/abstract). The paper describes these curves as the relative response per photon; as such, these filters are in the photon counting form so we need to convert it.

In [ ]:
# There are three files.
wise_1_filename = "./base/photometric_filters/filter_WISE_1.txt"
wise_2_filename = "./base/photometric_filters/filter_WISE_2.txt"
wise_3_filename = "./base/photometric_filters/filter_WISE_3.txt"
wise_4_filename = "./base/photometric_filters/filter_WISE_4.txt"
wise_filenames = [
    wise_1_filename,
    wise_2_filename,
    wise_3_filename,
    wise_4_filename,
]

for filedex in wise_filenames:
    # Reading the WISE data straight from the file.
    wise_data = np.genfromtxt(wise_1_filename, comments="#").T

    # We only need the data that is relevant to our filter tag. We skip over
    # any "masked" data.
    wavelength = np.array(wise_data[0])
    transmission = np.array(wise_data[1])
    uncertainty = np.array(wise_data[2])

    # The wavelength provided by the 2MASS files are in micrometers, we convert
    # to meters.
    wavelength_meter = wavelength / 1e6

    # We load the filter, leveraging the Lezargus container to do the
    # conversion for us.
    filter_container = lezargus.library.container.PhotometricVegaFilter.from_photon_transmission(
        wavelength=wavelength_meter,
        photon_transmission=transmission,
        wavelength_unit="m",
    )

    # The actual data we need to save.
    wavelength_column = astropy.table.Column(
        filter_container.wavelength, name="wavelength", unit="m", dtype=float
    )
    transmission_column = astropy.table.Column(
        filter_container.transmission_energy,
        name="transmission",
        unit="",
        dtype=float,
    )

    table_columns = [wavelength_column, transmission_column]
    wise_table = astropy.table.Table(table_columns)
    # Too much precision makes unnessary large file sizes.
    basename = lezargus.library.path.get_filename_without_extension(
        pathname=filedex
    )
    table_filename = f"./products/photometric_filters/{basename}.dat"
    wise_table.write(
        table_filename,
        format="ascii.mrt",
        formats={keydex: "%.7e" for keydex in wise_table.keys()},
        overwrite=True,
    )